In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install scikit-surprise

     |████████████████████████████████| 6.5MB 2.8MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1673616 sha256=9e5e3ddb6980ae08904c587f368bd424b848694aed6d221699b2c9f4f3fd8d54
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [ ]:
from __future__ import (absolute_import, division, print_function,unicode_literals)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate, train_test_split

In [ ]:
movies = pd.read_csv('/content/drive/My Drive/FinalYearProj-2020/DataSets/Movielens/movies.csv')
ratings = pd.read_csv('/content/drive/My Drive/FinalYearProj-2020/DataSets/Movielens/ratings.csv')

In [ ]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
movieRatings=pd.merge(movies,ratings,on="movieId")
movieRatings.head(10)

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,17,4.5,1305696483
5,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,18,3.5,1455209816
6,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,4.0,965705637
7,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,21,3.5,1407618878
8,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,27,3.0,962685262
9,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,31,5.0,850466616


In [ ]:
movieRatings= movieRatings.drop(["genres","timestamp"],axis=1)

In [ ]:
movieRatings

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0
100832,193583,No Game No Life: Zero (2017),184,3.5
100833,193585,Flint (2017),184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5


In [ ]:
movieRatings= movieRatings.dropna()
movieRatings

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5
...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0
100832,193583,No Game No Life: Zero (2017),184,3.5
100833,193585,Flint (2017),184,3.5
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5


In [ ]:
movie_ratingCount = (movieRatings.groupby(by=['movieId'])['rating'].count().reset_index().rename(columns = { 'rating':'totalRatingCount'})[['movieId','totalRatingCount']])
movie_ratingCount

,movieId,totalRatingCount
0,1,215
1,2,110
2,3,52
3,4,7
4,5,49
...,...,...
9719,193581,1
9720,193583,1
9721,193585,1
9722,193587,1


In [ ]:
rating_with_totalRatingCount = movieRatings.merge(movie_ratingCount, on = 'movieId')
rating_with_totalRatingCount

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215
...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),184,4.0,1
100832,193583,No Game No Life: Zero (2017),184,3.5,1
100833,193585,Flint (2017),184,3.5,1
100834,193587,Bungo Stray Dogs: Dead Apple (2018),184,3.5,1


In [ ]:
#giving some popularity thresold to discard less popular books
popularity_thresold = 50
rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount >= @popularity_thresold')
rating_popular_movie

,movieId,title,userId,rating,totalRatingCount
0,1,Toy Story (1995),1,4.0,215
1,1,Toy Story (1995),5,4.0,215
2,1,Toy Story (1995),7,4.5,215
3,1,Toy Story (1995),15,2.5,215
4,1,Toy Story (1995),17,4.5,215
...,...,...,...,...,...
98310,122904,Deadpool (2016),561,2.0,54
98311,122904,Deadpool (2016),586,4.0,54
98312,122904,Deadpool (2016),596,4.0,54
98313,122904,Deadpool (2016),599,3.5,54


Data Sanitization is complete.

---


Here I used only popularity thresold because I find that it is reducing the size more than 50%, too short dataset might overfit the model

In [ ]:
reader=Reader(rating_scale=(1,5))

In [ ]:
readerData = rating_popular_movie.iloc[:,[2,0,3]]
readerData

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
...,...,...,...
98310,561,122904,2.0
98311,586,122904,4.0
98312,596,122904,4.0
98313,599,122904,3.5


In [ ]:
data = Dataset.load_from_df(readerData,reader)

In [ ]:
data

Training Testing Data Split

In [ ]:
#keeping testing set as 20%
trainset, testset=train_test_split(data, test_size=0.2)

In [ ]:
trainset

In [ ]:
testset

[(17, 2571, 5.0),
 (569, 165, 5.0),
 (475, 109487, 4.0),
 (376, 586, 2.0),
 (538, 4018, 3.0),
 (480, 288, 2.5),
 (387, 3421, 4.0),
 (292, 69122, 3.5),
 (303, 6333, 5.0),
 (298, 3994, 1.0),
 (328, 1208, 4.0),
 (480, 3527, 2.5),
 (182, 4993, 3.0),
 (330, 2502, 5.0),
 (166, 2959, 4.5),
 (574, 47, 5.0),
 (168, 912, 4.5),
 (305, 1387, 3.5),
 (274, 4262, 5.0),
 (483, 2985, 2.5),
 (462, 589, 2.5),
 (274, 2006, 2.5),
 (480, 2947, 3.0),
 (108, 1079, 5.0),
 (141, 1517, 3.0),
 (500, 595, 4.0),
 (318, 4886, 3.5),
 (466, 41566, 4.0),
 (386, 34, 4.0),
 (135, 1291, 4.0),
 (577, 110, 4.0),
 (187, 2997, 5.0),
 (213, 908, 4.0),
 (334, 47, 2.0),
 (305, 2115, 1.0),
 (359, 1240, 2.5),
 (274, 3052, 4.5),
 (599, 2353, 2.5),
 (507, 377, 5.0),
 (489, 5816, 3.0),
 (305, 63082, 5.0),
 (89, 4973, 2.0),
 (500, 2683, 3.0),
 (274, 1136, 3.5),
 (290, 11, 4.0),
 (471, 6539, 3.5),
 (212, 1207, 3.5),
 (182, 3114, 4.0),
 (132, 1, 2.0),
 (432, 44191, 4.0),
 (312, 2012, 4.0),
 (13, 3793, 4.0),
 (274, 69844, 3.5),
 (554, 26

In [ ]:
from surprise import SVDpp,SlopeOne,NMF,NormalPredictor,KNNBaseline,KNNBasic,KNNWithMeans,KNNWithZScore,BaselineOnly,CoClustering

In [ ]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,0.829981,33.148544,1.529443
KNNBaseline,0.834038,0.079695,2.023131
SlopeOne,0.842964,0.114433,1.098167
SVD,0.843003,1.553975,0.108867
BaselineOnly,0.843372,0.027396,0.087635
KNNWithMeans,0.844645,0.062949,1.670042
KNNWithZScore,0.844995,0.086529,1.839725
NMF,0.865122,1.547722,0.098112
CoClustering,0.882474,0.381489,0.066991


In [ ]:
from surprise import accuracy
acc=accuracy.rmse(predictions)

RMSE: 0.8433


In [ ]:
acc

0.84331855536119

Comparative Study

In [ ]:
comp_study=pd.DataFrame(columns=["Algorithm","Accuracy"])
c=0
alg_names=["SVD","SVDpp","SlopeOne","NMF","NormalPredictor","KNNBaseline","KNNBasic","KNNWithMeans","KNNWithZScore","BaselineOnly","CoClustering"]
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    model=algorithm
    model.fit(trainset)
    pred=model.test(testset)
    acc=accuracy.rmse(pred)
    # print(algorithm.instance)
    # tmp=pd.DataFrame({algorithm:acc})
    comp_study=comp_study.append([{"Algorithm":alg_names[c],"Accuracy":acc}])
    c+=1
comp_study.set_index('Algorithm').sort_values("Accuracy",ascending=False)
    


RMSE: 0.8435
RMSE: 0.8249
RMSE: 0.8456
RMSE: 0.8571
RMSE: 1.3447
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8340
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8843
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8439
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8438
Estimating biases using als...
RMSE: 0.8464
RMSE: 0.8781


,Accuracy
Algorithm,
NormalPredictor,1.344720
KNNBasic,0.884267
CoClustering,0.878083
NMF,0.857072
BaselineOnly,0.846408
SlopeOne,0.845620
KNNWithMeans,0.843905
KNNWithZScore,0.843767
SVD,0.843493


In [ ]:
#training your trainset with 6 features using SVD
modelSVD=SVD(n_factors=6)
trainedData=modelSVD.fit(trainset)

In [ ]:
print(trainedData)

In [ ]:
modelSVD.fit(trainset)

In [ ]:
res=cross_validate(modelSVD,data,measures=["RMSE"],cv=5, verbose=True)

Evaluating RMSE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8388  0.8352  0.8357  0.8330  0.8379  0.8361  0.0020  
Fit time          0.56    0.56    0.55    0.55    0.56    0.55    0.00    
Test time         0.06    0.05    0.05    0.13    0.05    0.07    0.03    


In [ ]:
pred=modelSVD.test(testset)

In [ ]:
from surprise import accuracy
accuracy.rmse(pred)

RMSE: 0.8406


0.8405516178444652